In [1]:
import pandas as pd
import pandas_datareader as data
import numpy as np
from matplotlib.pyplot import plt

In [2]:
#get stock data
start='2012-1-1'
end='2022-1-3'
df=data.DataReader('AAPL','yahoo',start,end)
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2012-01-03,14.732143,14.607143,14.621429,14.686786,302220800.0,12.591942
2012-01-04,14.810000,14.617143,14.642857,14.765714,260022000.0,12.659609
2012-01-05,14.948214,14.738214,14.819643,14.929643,271269600.0,12.800158
2012-01-06,15.098214,14.972143,14.991786,15.085714,318292800.0,12.933966
2012-01-09,15.276786,15.048214,15.196429,15.061786,394024400.0,12.913446


In [3]:
df=df.reset_index()
df=df.drop(['Date','Adj Close'],axis=1)
df.head()

,High,Low,Open,Close,Volume
0,14.732143,14.607143,14.621429,14.686786,302220800.0
1,14.810000,14.617143,14.642857,14.765714,260022000.0
2,14.948214,14.738214,14.819643,14.929643,271269600.0
3,15.098214,14.972143,14.991786,15.085714,318292800.0
4,15.276786,15.048214,15.196429,15.061786,394024400.0


In [4]:
ma100=df.Close.rolling(100).mean()
ma100

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
          ...   
2513    153.6667
2514    153.9991
2515    154.3202
2516    154.6399
2517    155.0014
Name: Close, Length: 2518, dtype: float64

In [5]:
ma200=df.Close.rolling(200).mean()
ma200

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
2513    143.13220
2514    143.40125
2515    143.66845
2516    143.95365
2517    144.26375
Name: Close, Length: 2518, dtype: float64

In [6]:
df.shape

(2518, 5)

In [7]:
#spliting data into Training and Testing
data_training=pd.DataFrame(df['Close'][0:int(len(df)*0.70)])
data_testing=pd.DataFrame(df['Close'][int(len(df)*0.70):int(len(df))])
print(data_training.shape)
print(data_testing.shape)

(1762, 1)
(756, 1)


In [9]:
data_training.head()

,Close
0,14.686786
1,14.765714
2,14.929643
3,15.085714
4,15.061786


In [10]:
data_testing.head()

,Close
1762,37.064999
1763,36.982498
1764,37.687500
1765,38.327499
1766,38.450001


In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))


In [15]:
data_training_array=scaler.fit_transform(data_training)
data_training_array

array([[0.01677525],
       [0.01856622],
       [0.02228596],
       ...,
       [0.57834128],
       [0.57936234],
       [0.49012933]])

In [17]:
x_train=[]
y_train=[]

for i in range(100,data_training_array.shape[0]):
    x_train.append(data_training_array[i-100:i])
    y_train.append(data_training_array[i,0])
x_train,y_train=np.array(x_train),np.array(y_train)

In [19]:
x_train.shape

(1662, 100, 1)

In [18]:
#ml model

In [20]:
from keras.layers import Dense,Dropout,LSTM
from keras.models import Sequential

In [21]:
model=Sequential()
model.add(LSTM(units=50,activation='relu',return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))



model.add(LSTM(units=60,activation='relu',return_sequences=True))
model.add(Dropout(0.3))



model.add(LSTM(units=80,activation='relu',return_sequences=True))
model.add(Dropout(0.4))




model.add(LSTM(units=120,activation='relu',return_sequences=True))
model.add(Dropout(0.5))

model.add(Dense(units=1))

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 50)           10400     
                                                                 
 dropout (Dropout)           (None, 100, 50)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100, 60)           26640     
                                                                 
 dropout_1 (Dropout)         (None, 100, 60)           0         
                                                                 
 lstm_2 (LSTM)               (None, 100, 80)           45120     
                                                                 
 dropout_2 (Dropout)         (None, 100, 80)           0         
                                                                 
 lstm_3 (LSTM)               (None, 100, 120)          9

In [23]:
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(x_train,y_train,epochs=50)

Epoch 1/50
52/52 [==============================] - 14s 182ms/step - loss: 0.0753
Epoch 2/50
52/52 [==============================] - 10s 195ms/step - loss: 0.0593
Epoch 3/50
52/52 [==============================] - 10s 188ms/step - loss: 0.0582
Epoch 4/50
52/52 [==============================] - 10s 195ms/step - loss: 0.0574
Epoch 5/50
52/52 [==============================] - 12s 223ms/step - loss: 0.0571
Epoch 6/50
52/52 [==============================] - 11s 217ms/step - loss: 0.0569
Epoch 7/50
52/52 [==============================] - 12s 227ms/step - loss: 0.0569
Epoch 8/50
52/52 [==============================] - 11s 214ms/step - loss: 0.0563
Epoch 9/50
52/52 [==============================] - 12s 228ms/step - loss: 0.0560
Epoch 10/50
52/52 [==============================] - 10s 195ms/step - loss: 0.0558
Epoch 11/50
52/52 [==============================] - 10s 198ms/step - loss: 0.0556
Epoch 12/50
52/52 [==============================] - 10s 197ms/step - loss: 0.0563
Epoch 13/50
5

In [1]:
model.save('kerasml.h5')

NameError: name 'model' is not defined